In [1]:
import folium
import pandas as pd
# Import file
%store -r data_sampah
%store -r koordinat_BDG

# Mengubah kolom tanggal menjadi tipe data datetime
data_sampah['tanggal'] = pd.to_datetime(data_sampah['tanggal'], format='%m/%d/%Y')

# Membuat kolom baru untuk mengelompokkan kolom tanggal berdasarkan bulan
data_sampah['tahun'] = data_sampah['tanggal'].dt.to_period('Y')

# Mengelompokkan berdasarkan bulan dan kecamatan, kemudian menghitung jumlah sampah
grup_tahun_kecamatan = data_sampah.groupby(['tahun', 'kecamatan']).agg({
    'jumlah_sampah': 'sum'
}).reset_index()

# Menggabungkan dengan data koordinat dan data sampah
data_gabung = pd.merge(grup_tahun_kecamatan, koordinat_BDG, on='kecamatan')

# Memasukan longitude dan latitud(rata rata supaya berada ditengah)
bandung_map = folium.Map(location=[koordinat_BDG['Lintang Selatan'].mean(), koordinat_BDG['Bujur Timur'].mean()], zoom_start=13)

# Perulangan untuk mengecek setiap kolom
for index, row in data_gabung.iterrows():
    # Menentukan warna marker berdasarkan jumlah sampah
    if row['jumlah_sampah'] > 1500:
        color = 'red'
    elif row['jumlah_sampah'] > 1000:
        color = 'orange'
    else:
        color = 'green'

    # Menambahkan marker untuk setiap baris dengan warna berdasarkan jumlah sampah
    popup_text = f"Jumlah Sampah Kecamatan {row['kecamatan']} {row['jumlah_sampah']} Ton per tahun "
    marker = folium.Marker(location=[row['Lintang Selatan'], row['Bujur Timur']], popup=popup_text, icon=folium.Icon(color=color))
    marker.add_to(bandung_map)

# Menampilkan peta di notebook
from IPython.display import display
display(bandung_map)
